In [ ]:
#default_exp enumify

# Enumify

> The magic decorator for Enums

In [ ]:
#hide
from nbdev.showdoc import *

## Enumify - 

The goal with `Enumify` is two fold:

1. Quickly generate `Enums` who have members of their names without needing to repeat itself
2. Be able to document these `Enums` as well, when the matter is needed. 

These behaviors slightly change based on the expected end-environment, so be sure to read carefully which annotation you should use

In [ ]:
#hide
from fastcore.test import test_eq

In [ ]:
#export
from enumify.typing import Mem, Member
from enumify.enum import FunctionalEnum, DocumentedEnum
from functools import partial

In [ ]:
#export
def assign_annotations(cls, interactive:bool=True):
    """
    Creates a FunctionalEnum or DocumentedEnum based on annotations and asserts in `cls`
    
    Args:
      cls:
        A general class to be converted to an Enum
      interactive (`bool`), default `False`
        Whether the enum is being made for a Jupyter environment or not,
        and should set `__doc__`
    """
    # First filter out all but what we need: doc, annotations, and set members
    d = dict(cls.__dict__)
    _keep = ["__doc__", "__annotations__"]
    for key in list(d):
        if key.startswith("_") and key not in _keep:
            d.pop(key, None)
            
    names = [] # Names for our enum
    keys = []
    for name, typ in d["__annotations__"].items():
        if not typ == Mem or not typ == Member:
            continue
        value = name.lower()
        if not interactive:
            names.append([name, value])
        else:
            names.append([name, (value, getattr(cls, name, "An enumeration."))])
        keys.append(name)
        
    for name in d:
        if name not in keys and not name.startswith("_"):
            if not interactive:
                names.append([name, getattr(cls, name)])
            else:
                value = getattr(cls, name)
                if isinstance(value, tuple):
                    value, doc = value
                else:
                    doc = "An enumeration."
                names.append([name, (value, doc)])
            keys.append(name)
    if not interactive:
        new_cls = FunctionalEnum(value=cls.__name__, names=names)
        new_cls.__doc__ = cls.__doc__
    else:
        new_cls = DocumentedEnum(value=cls.__name__, names=names)
        new_cls.__doc__ = cls.__doc__
    return new_cls

In [ ]:
class DaysOfWeek:
    MONDAY:Member = "First day of the week"
    WEDNESDAY = "Wed", "Third day of the week"
    THURSDAY:int = 0

NewAnnotation = assign_annotations(DaysOfWeek, interactive=True)
test_eq(NewAnnotation.MONDAY, "monday")
test_eq(NewAnnotation.MONDAY.__doc__, "First day of the week")
test_eq(NewAnnotation.WEDNESDAY, "Wed")
test_eq(NewAnnotation.WEDNESDAY.__doc__, "Third day of the week")
test_eq(NewAnnotation.THURSDAY, 0)
test_eq(NewAnnotation.THURSDAY.__doc__, "An enumeration.")

In [ ]:
class DaysOfWeek:
    MONDAY:Member
    WEDNESDAY = "Wed"
    THURSDAY:int = 0

NewAnnotation = assign_annotations(DaysOfWeek, interactive=False)
test_eq(NewAnnotation.MONDAY, "monday")
test_eq(NewAnnotation.MONDAY.__doc__, None)
test_eq(NewAnnotation.WEDNESDAY, "Wed")
test_eq(NewAnnotation.WEDNESDAY.__doc__, None)
test_eq(NewAnnotation.THURSDAY, 0)
test_eq(NewAnnotation.THURSDAY.__doc__, None)

In [ ]:
class DaysOfWeek:
    MONDAY:Member
    WEDNESDAY:Member
    THURSDAY:Member

NewAnnotation = assign_annotations(DaysOfWeek, interactive=False)
test_eq(NewAnnotation.MONDAY, "monday")
test_eq(NewAnnotation.MONDAY.__doc__, None)
test_eq(NewAnnotation.WEDNESDAY, "wednesday")
test_eq(NewAnnotation.WEDNESDAY.__doc__, None)
test_eq(NewAnnotation.THURSDAY, "thursday")
test_eq(NewAnnotation.THURSDAY.__doc__, None)

In [ ]:
#export
def enumify(cls=None, interactive:bool=True):
    """
    A decorator to turn `cls` into an Enum class with member values as property names, and potentially with documentation
    
    If the end result should be used in a notebook, pass `interactive=True` to have the 
    documentation linked to the enum. Otherwise `False`.
    
    Should be documented with the `Member` type with the following annotation if the end result should be in jupyter:
    ```python
    from enumify.typing import Member
    @enumify
    class MyClass:
      NAME:Member
      name_three:Member = "some value", "Some documentation"
    ```
    And the following if targeting environments such as VSCode:
    ```python
    from enumify.typing import Member
    @enumify(interactive=False)
    class MyClass:
      NAME:Member
      name_three:Member = "some value"
      "Some documentation"
    ```
    
    Can also use the shorthand `Mem` type.
    """
    def wrap(cls, interactive): return assign_annotations(cls, interactive)
    if cls is None:
        return partial(enumify, interactive=interactive)
    return wrap(cls, interactive)

In [ ]:
@enumify
class DaysOfWeek:
    MONDAY:Member = "First day of the week"
    WEDNESDAY = "Wed", "Third day of the week"
    THURSDAY:int = 0
    
test_eq(DaysOfWeek.MONDAY, "monday")
test_eq(DaysOfWeek.MONDAY.__doc__, "First day of the week")
test_eq(DaysOfWeek.WEDNESDAY, "Wed")
test_eq(DaysOfWeek.WEDNESDAY.__doc__, "Third day of the week")
test_eq(DaysOfWeek.THURSDAY, 0)
test_eq(DaysOfWeek.THURSDAY.__doc__, "An enumeration.")

In [ ]:
@enumify(interactive=False)
class DaysOfWeek:
    MONDAY:Member
    WEDNESDAY = "Wed"
    THURSDAY:int = 0
    
test_eq(DaysOfWeek.MONDAY, "monday")
test_eq(DaysOfWeek.MONDAY.__doc__, None)
test_eq(DaysOfWeek.WEDNESDAY, "Wed")
test_eq(DaysOfWeek.WEDNESDAY.__doc__, None)
test_eq(DaysOfWeek.THURSDAY, 0)
test_eq(DaysOfWeek.THURSDAY.__doc__, None)